In [9]:
import grequests
from random import randint
from lxml.html import fromstring

### Algorithm for generating random CNPJs

In [10]:
def validate_digit(mult_array, digits_array):
    sum_digits = sum(map(lambda x,y: x*y, mult_array, digits_array))
    dig = sum_digits % 11
    
    if dig < 2:
        dig = 0
    else:
        dig = 11 - dig

    return dig


def generate_cnpj():
    cnpj_12d = str(randint(1000,1e8)).zfill(8) + '0001'
    cnpj_int_l = [int(d) for d in list(cnpj_12d)]

    first_dig = validate_digit([5,4,3,2,9,8,7,6,5,4,3,2], cnpj_int_l)
    sec_dig = validate_digit([6,5,4,3,2,9,8,7,6,5,4,3,2], cnpj_int_l + [first_dig] )
    return cnpj_12d + str(first_dig) + str(sec_dig)


In [11]:
#Write each random CNPJ to a text file. These file will be used to fetch data from the receitaWS API

n = 10000
list_cnpjs = [generate_cnpj() for i in range(n)]

with open('list_cnpjs.txt', 'w') as f:
    for cnpj in list_cnpjs:
        f.write(f'{cnpj}\n')

### Free Proxies

In [12]:
import requests

In [13]:
def fetch_proxies():
    response = requests.get('https://free-proxy-list.net/')
    parser = fromstring(response.text)
    proxies = set()
    for i in parser.xpath('//tbody/tr')[:300]: # this site provides up to 300 free proxies
        # We do not want transparent proxies, since they do not work to bypass APIs
        if i.xpath('.//td[5][not(contains(text(),"transparent"))]'):
            # IP + Port
            proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
            proxies.add(proxy)
    return proxies

In [14]:
l1 = fetch_proxies()
len(l1)



270

In [15]:
def exception_handler(request, exception):
    print("Request failed", exception)

def generate_requests(cnpjs, proxies):
    ## Create the list for parallel connections through free proxies
    return [grequests.get(f'https://www.receitaws.com.br/v1/cnpj/{cnpj}', 
                          proxies = {
                            "http": f'http://{proxy}', 
                            "https": f'http://{proxy}' 
                          },
                          timeout=10) 
            for cnpj, proxy in zip(cnpjs, proxies)]


In [16]:
reqs = generate_requests(list_cnpjs[:15], list(l1)[40:55])
resp = grequests.map(reqs)

In [17]:
resp

[None,
 None,
 None,
 <Response [200]>,
 None,
 None,
 None,
 None,
 None,
 <Response [200]>,
 <Response [200]>,
 None,
 None,
 None,
 None]